In [3]:
import pandas as pd
import numpy as np
from metar import Metar
import datetime as dt
import fuzzymatcher
import os
from geopy import distance
from geopy import Point
import glob
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from dateutil.parser import parse as parse_date
pd.set_option('display.max_columns', 999)

In [4]:
# load data frame
fw = pd.read_csv("..//Datasets/NEW/FW_with_airports.csv")
fw.head()

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,arrival,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,iata_code_origin,AIRPORT_NAME_origin,CITY_NAME_origin,COUNTRY_NAME_origin,STATE_NAME_origin,origin_state,ICAO_code_destination,iata_code_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,destination_state
0,N100KB,20-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",04:57PM CST,05:44PM CST,0:46,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,46.772701,-100.746002,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND
1,N100KB,20-Feb-21,BE9L,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,01:36PM CST,02:27PM CST,0:51,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",46.772701,-100.746002,48.260833,-103.751111,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND
2,N100KB,18-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",08:20AM CST,08:53AM CST,0:32,105.704153,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,46.772701,-100.746002,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND
3,N100KB,15-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",02:15PM CST,02:49PM CST,0:33,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,48.259399,-101.279999,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND
4,N100KB,15-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,08:00AM CST,08:32AM CST,0:32,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.260833,-103.751111,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND


In [295]:
fw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23160 entries, 0 to 23159
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tail_number               23160 non-null  object 
 1   date                      23160 non-null  object 
 2   aircraft                  22966 non-null  object 
 3   origin                    23160 non-null  object 
 4   origin_location           23160 non-null  object 
 5   destination               23159 non-null  object 
 6   destination_location      23159 non-null  object 
 7   departure                 23160 non-null  object 
 8   arrival                   22734 non-null  object 
 9   duration                  22711 non-null  object 
 10  Owner                     23160 non-null  object 
 11  origin_Latitude           23001 non-null  float64
 12  origin_Longitude          23001 non-null  float64
 13  destination_Latitude      23002 non-null  float64
 14  destin

In [296]:
fw.shape

(23160, 29)

In [297]:
# the number of the same origin and destination locations
fw.loc[(fw.ICAO_code_origin == fw.ICAO_code_destination) | (fw.iata_code_origin == fw.iata_code_destination)].shape

(833, 29)

In [298]:
# observe these records  
fw.loc[(fw.ICAO_code_origin == fw.ICAO_code_destination) | (fw.iata_code_origin == fw.iata_code_destination), 
       ['ICAO_code_origin', 'iata_code_origin', 'ICAO_code_destination', 'iata_code_destination']].head()

,ICAO_code_origin,iata_code_origin,ICAO_code_destination,iata_code_destination
32,KBIS,BIS,KBIS,BIS
33,KBIS,BIS,KBIS,BIS
41,KMOT,MOT,KMOT,MOT
75,KMOT,MOT,KMOT,MOT
94,KDAL,DAL,KDAL,DAL


In [299]:
# drop all the same origin and destination locations
fw.loc[(fw.ICAO_code_origin == fw.ICAO_code_destination) | (fw.iata_code_origin == fw.iata_code_destination), fw.columns[15:21]] = np.nan
fw.loc[(fw.ICAO_code_origin.isna()) & (fw.iata_code_origin.isna()), fw.columns[22:-1]] = np.nan

In [300]:
# calculate the distance
for i in range(len(fw)):
    try:
        p1 = Point(str(fw.origin_Latitude[i]) + ' ' + str(fw.origin_Longitude[i]))
        p2 = Point(str(fw.destination_Latitude[i]) + ' ' + str(fw.destination_Longitude[i]))
    
        fw.loc[i, 'distance_mi'] = distance.distance(p1,p2).miles
    except:
        pass

In [301]:
# convert date to datetime
fw.date = pd.to_datetime(fw.date, format='%d-%b-%y')

In [302]:
# split time zone and assign to new variable for departure
fw['dep_time_zone'] = fw.departure.str.split(" ", n = 1, expand = True)[1]

# remove time zome from departure
fw.departure = fw.departure.str.split(" ", n = 1, expand = True)[0]

# split time zone and assign to new variable for arrival
fw['arr_time_zone'] = fw.arrival.str.split(" ", n = 1, expand = True)[1].str.split(" ", n = 1, expand = True)[0]

# remove time zome from arrival
fw.arrival = fw.arrival.str.split(" ", n = 1, expand = True)[0]

In [303]:
# rearrange data for the convenience 
fw = fw.iloc[:,:8].join([fw.iloc[:,-2], fw.iloc[:,8], fw.iloc[:,-1], fw.iloc[:,9], fw.iloc[:,-3], fw.iloc[:,10:29]])

In [304]:
# convert departure time to datetime format
fw.departure = pd.to_datetime(fw.departure, errors='coerce').dt.time

# convert arrival time to datetime format
fw.arrival = pd.to_datetime(fw.arrival, errors='coerce').dt.time

# drop null values
fw = fw.dropna(subset=['departure', 'arrival'])

#### Time zone differences
- **PST**: UTC−08, **CST**: UTC−06, **CAT**: UTC+02 
- **MST**: UTC−07, **AST**: UTC−04, **ES**T: UTC−05
- **HST**: UTC−10, **NST**: UTC−03:30, **WAT**: UTC+01   
- **EET**: UTC+02, **EAT**: UTC+03, **AEDT**: UTC+11     
- **CE**T: UTC+01, **AKST**: UTC−09, **JST**: UTC+09, **ChST**: UTC+10  

In [305]:
# identify unique time zones for departure time
print(fw.dep_time_zone.sort_values(ascending=False).unique())

['WAT' 'PST' 'MST' 'HST' 'GMT' 'EST' 'EET' 'ChST' 'CST' 'AST' 'AKST'
 'AEDT' '-01' '+12' '+10' '+09' '+04' '+03']


In [306]:
# convert departure time to UTC
fw['dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'PST','departure']\
                       .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=8)).time())

fw.loc[fw.dep_time_zone == 'WAT','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'WAT','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=1)).time())

fw.loc[fw.dep_time_zone == 'ChST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'ChST','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=10)).time())

fw.loc[fw.dep_time_zone == 'MST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'MST','departure']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=7)).time())

# fw.loc[fw.dep_time_zone == 'AEDT','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'AEDT','departure']\
#                                                      .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=11)).time())

fw.loc[fw.dep_time_zone == 'HST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'HST','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=10)).time())

fw.loc[fw.dep_time_zone == 'GMT','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'GMT','departure']

fw.loc[fw.dep_time_zone == 'EST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'EST','departure']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=5)).time())

fw.loc[fw.dep_time_zone == 'EET','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'EET','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=2)).time())

fw.loc[fw.dep_time_zone == 'EAT','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'EAT','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=3)).time())

fw.loc[fw.dep_time_zone == 'CST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'CST','departure']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=6)).time())
# fw.loc[fw.dep_time_zone == 'CET','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'CET','departure']\
#                                                   .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=1)).time())

fw.loc[fw.dep_time_zone == 'AST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'AST','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=4)).time())

fw.loc[fw.dep_time_zone == 'AKST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'AKST','departure']\
                                                   .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=9)).time())

In [307]:
# identify unique time zones for arrival time
print(fw.arr_time_zone.sort_values(ascending=False).unique())

['WAT' 'PST' 'MST' 'HST' 'GMT' 'EST' 'EET' 'ChST' 'CST' 'AST' 'AKST'
 'AEDT' '-01' '+12' '+10' '+09' '+06' '+04' '+03']


In [308]:
# convert arrival time to UTC
fw['arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'PST','arrival']\
                       .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=8)).time())

fw.loc[fw.arr_time_zone == 'NST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'NST','arrival']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=3, minutes=30)).time())

fw.loc[fw.arr_time_zone == 'MST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'MST','arrival']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=7)).time())

fw.loc[fw.arr_time_zone == 'WAT','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'WAT','arrival']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=1)).time())

fw.loc[fw.arr_time_zone == 'HST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'HST','arrival']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=10)).time())

fw.loc[fw.arr_time_zone == 'GMT','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'GMT','arrival']

fw.loc[fw.arr_time_zone == 'EST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'EST','arrival']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=5)).time())

fw.loc[fw.arr_time_zone == 'EET','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'EET','arrival']\
                                                   .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=2)).time())

# fw.loc[fw.arr_time_zone == 'AEDT','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'AEDT','arrival']\
#                                                   .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=11)).time())

fw.loc[fw.arr_time_zone == 'CST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'CST','arrival']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=6)).time())

fw.loc[fw.arr_time_zone == 'ChST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'ChST','arrival']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=10)).time())

fw.loc[fw.arr_time_zone == 'CAT','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'CAT','arrival']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=2)).time())

fw.loc[fw.arr_time_zone == 'AST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'AST','arrival']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=4)).time())

fw.loc[fw.arr_time_zone == 'AKST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'AKST','arrival']\
                                                   .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=9)).time())

In [17]:
# # =======================================================================================
# #                       METAR data merging preparation for Origin
# # =======================================================================================
fw_origin = fw.loc[:,['tail_number','ICAO_code_origin', 'date', 'dep_UTC_time']]
fw_origin.head()

,tail_number,ICAO_code_origin,date,dep_UTC_time
0,N100KB,KXWA,2021-02-20,22:57:00
1,N100KB,KBIS,2021-02-20,19:36:00
2,N100KB,KMOT,2021-02-18,14:20:00
3,N100KB,KXWA,2021-02-15,20:15:00
4,N100KB,KMOT,2021-02-15,14:00:00


In [18]:
# replace dual ICAO code
fw_origin['spare'] = fw_origin.ICAO_code_origin.str.split(" / ", expand=True, n=1)[1]
fw_origin.loc[:,'ICAO_code_origin'] = fw_origin.ICAO_code_origin.str.split(" / ", expand=True, n=1)[0]
fw_origin.loc[fw_origin.spare.notnull(),'ICAO_code_origin'] = fw_origin.loc[fw_origin.spare.notnull(), 'spare']
fw_origin = fw_origin.drop('spare', axis=1)

In [19]:
# drop null values
fw_origin = fw_origin.dropna()

In [20]:
# sort by tail number 
fw_origin = fw_origin.sort_values('tail_number').reset_index(drop=True)

In [21]:
fw_origin.head()

,tail_number,ICAO_code_origin,date,dep_UTC_time
0,N100KB,KXWA,2021-02-20,22:57:00
1,N100KB,KXWA,2020-12-14,20:46:00
2,N100KB,KMOT,2020-12-20,14:48:00
3,N100KB,KFAR,2020-12-20,17:11:00
4,N100KB,KMOT,2020-12-20,01:40:00


In [22]:
# =======================================================================================
#                               METAR data scraping for Origin
# =======================================================================================

# create containers
tail_number = []
ICAO_origin = []
date = []
METAR = []

for j in range(0,len(fw_origin)-1):
    url = "http://www.ogimet.com/display_metars2.php?lang=en&lugar=" + fw_origin.ICAO_code_origin.iloc[j] \
      + "&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=" + str(fw_origin.date.dt.year.iloc[j]) \
      + "&mes=" + str(fw_origin.date.dt.month.iloc[j]) + "&day=" + str(fw_origin.date.dt.day.iloc[j]) \
      +"&hora=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[j]) + "&anof=" + str(fw_origin.date.dt.year.iloc[j]) \
      + "&mesf="+ str(fw_origin.date.dt.month.iloc[j]) + "&dayf=" + str(fw_origin.date.dt.day.iloc[j]) \
    + "&horaf=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[j] + 1) + "&minf=59&send=send"

    # BeautifulSoup Magic
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, 'html.parser')
    
    if fw_origin.tail_number.iloc[j] == fw_origin.tail_number.iloc[j+1]:
        for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
            tail_number += [fw_origin.tail_number.iloc[j]]
            ICAO_origin += [fw_origin.ICAO_code_origin.iloc[j]]
            date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
            METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]

# if reaches the end
    if j == len(fw_origin)-2:
        url = "http://www.ogimet.com/display_metars2.php?lang=en&lugar=" + fw_origin.ICAO_code_origin.iloc[-1] \
          + "&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=" + str(fw_origin.date.dt.year.iloc[-1]) \
          + "&mes=" + str(fw_origin.date.dt.month.iloc[-1]) + "&day=" + str(fw_origin.date.dt.day.iloc[-1]) \
          +"&hora=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[-1]) + "&anof=" + str(fw_origin.date.dt.year.iloc[-1]) \
          + "&mesf="+ str(fw_origin.date.dt.month.iloc[-1]) + "&dayf=" + str(fw_origin.date.dt.day.iloc[-1]) \
          + "&horaf=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[-1] + 1) + "&minf=59&send=send"
        
        # BeautifulSoup Magic
        uClient = uReq(url)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, 'html.parser')
        
        for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
            tail_number += [fw_origin.tail_number.iloc[-1]]
            ICAO_origin += [fw_origin.ICAO_code_origin.iloc[-1]]
            date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
            METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
        
        pd.DataFrame(list(zip(tail_number, ICAO_origin, date, METAR)), columns= ['tail_number', 'ICAO_origin', 'date', 'METAR'])\
        .to_csv('../Datasets/METAR/FW/origin/' + fw_origin.tail_number.iloc[-1] + '.csv', index=False) 
        
    else:
        for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
            tail_number += [fw_origin.tail_number.iloc[j]]
            ICAO_origin += [fw_origin.ICAO_code_origin.iloc[j]]
            date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
            METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
        
        pd.DataFrame(list(zip(tail_number, ICAO_origin, date, METAR)), columns= ['tail_number', 'ICAO_origin', 'date', 'METAR'])\
        .to_csv('../Datasets/METAR/FW/origin/' + fw_origin.tail_number.iloc[j] + '.csv', index=False) 
        
        tail_number = []
        ICAO_origin = []
        date = []
        METAR = []

In [23]:
# # concat all files
# all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]

# #combine all files in the list
# combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# #export to csv
# combined_csv.to_csv("FW_METAR_origin.csv", index=False, encoding='utf-8-sig')

In [414]:
# # =======================================================================================
# #                               METAR data merging for Origin
# # =======================================================================================
# load the origin METAR dataframe
metar_origin = pd.read_csv("../Datasets/METAR/FW/origin/FW_METAR_origin.csv")

In [415]:
metar_origin.head()

,tail_number,ICAO_origin,date,METAR
0,N783AL,KCOS,04/01/2021 19:54,METAR KCOS 041954Z 19007KT 10SM SCT250 08/M09 ...
1,N783AL,KCOS,04/01/2021 18:54,METAR KCOS 041854Z 13008KT 10SM SCT250 04/M08 ...
2,N783AL,KAPA,04/01/2021 17:53,METAR KAPA 041753Z 22010KT 10SM FEW100 FEW200 ...
3,N783AL,KAPA,04/01/2021 16:53,METAR KAPA 041653Z 23009KT 10SM FEW100 FEW200 ...
4,N783AL,KSBS,01/01/2021 22:55,METAR KSBS 012255Z AUTO 00000KT 10SM CLR M04/M...


In [416]:
# split time from the date and assign a new column
metar_origin['departure'] = metar_origin.date.str.split(" ", n=1, expand=True)[1]

# remove time from the date columns
metar_origin.date = metar_origin.date.str.split(" ", n=1, expand=True)[0]

In [418]:
# convert date to datetime
metar_origin.date  = pd.to_datetime(metar_origin.date, format='%d/%m/%Y')

# conver time to datetime
metar_origin.departure = pd.to_datetime(metar_origin.departure, format='%H:%M').dt.time

In [420]:
# convert to string fot the best matching
metar_origin.date = metar_origin.date.astype(str)
metar_origin.departure = metar_origin.departure.astype(str)

fw.date = fw.date.astype(str)
fw.dep_UTC_time = fw.dep_UTC_time.astype(str)

In [421]:
metar_origin.head()

,tail_number,ICAO_origin,date,METAR,departure
0,N783AL,KCOS,2021-01-04,METAR KCOS 041954Z 19007KT 10SM SCT250 08/M09 ...,19:54:00
1,N783AL,KCOS,2021-01-04,METAR KCOS 041854Z 13008KT 10SM SCT250 04/M08 ...,18:54:00
2,N783AL,KAPA,2021-01-04,METAR KAPA 041753Z 22010KT 10SM FEW100 FEW200 ...,17:53:00
3,N783AL,KAPA,2021-01-04,METAR KAPA 041653Z 23009KT 10SM FEW100 FEW200 ...,16:53:00
4,N783AL,KSBS,2021-01-01,METAR KSBS 012255Z AUTO 00000KT 10SM CLR M04/M...,22:55:00


In [315]:
# merge origin METAR data 
fw = fuzzymatcher.fuzzy_left_join(fw, metar_origin.add_suffix('_origin'), left_on = ['tail_number','ICAO_code_origin', 'date', 'dep_UTC_time'], 
                                  right_on = ['tail_number_origin','ICAO_origin_origin', 'date_origin', 'departure_origin']).iloc[:,3:]

In [316]:
# count of records where the ICAO_code_origin is null or not in metar_origin column
fw.loc[(~fw.ICAO_code_origin.isin(fw.ICAO_origin_origin)) | (fw.ICAO_code_origin.isna())].shape

(2967, 39)

In [317]:
# Observe where the ICAO_code_origin is null or not in metar_origin columns
fw.loc[(~fw.ICAO_code_origin.isin(fw.ICAO_origin_origin)) | (fw.ICAO_code_origin.isna()), ['ICAO_code_origin', 'ICAO_origin_origin']].head()

,ICAO_code_origin,ICAO_origin_origin
254,06D,KMOT
1346,S25,KMOT
1516,06D,KMOT
1792,NaN,KBIS
1798,NaN,KBIS


In [318]:
# Drop where the ICAO_code_origin is null or not in metar_origin
fw.loc[(~fw.ICAO_code_origin.isin(fw.ICAO_origin_origin)) | (fw.ICAO_code_origin.isna()), fw.columns[-5:39]] = np.nan

In [319]:
# number of mismached tail numbers
fw.loc[(fw.tail_number != fw.tail_number_origin) & (fw.tail_number_origin.notnull())].shape

(1300, 39)

In [320]:
# inspect this records
fw.loc[(fw.tail_number != fw.tail_number_origin) & (fw.tail_number_origin.notnull()),
       ['tail_number', 'ICAO_code_origin', 'date', 'dep_UTC_time', 'tail_number_origin', 'ICAO_origin_origin', 'date_origin','departure_origin']].head()

,tail_number,ICAO_code_origin,date,dep_UTC_time,tail_number_origin,ICAO_origin_origin,date_origin,departure_origin
6,N100KB,KMOT,2021-02-18,14:20:00,N986AC,KPIR,2021-02-18,20:53:00
4590,N149CF,KDAL,2021-03-07,13:10:00,N460EM,KDAL,2021-03-07,10:53:00
5619,N149CF,KGPM,2021-02-23,14:17:00,N545LM,PANC,2021-02-03,17:14:00
6116,N149CF,KSJT,2021-02-20,11:54:00,N495Y,KMKC,2021-02-20,11:54:00
6849,N149CF,KGPM,2021-02-19,17:49:00,N481HC,KIDA,2021-02-19,17:53:00


In [321]:
# drop this records
fw.loc[(fw.tail_number != fw.tail_number_origin) & (fw.tail_number_origin.notnull()), fw.columns[-5:39]] = np.nan

In [322]:
# number of mismached ICAO_origin_origin
fw.loc[(fw.ICAO_code_origin.notnull()) & (fw.ICAO_origin_origin.notnull()) & (fw.ICAO_code_origin != fw.ICAO_origin_origin)].shape

(821, 39)

In [323]:
# inspect this records
fw.loc[(fw.ICAO_code_origin.notnull()) & (fw.ICAO_origin_origin.notnull()) & (fw.ICAO_code_origin != fw.ICAO_origin_origin),
       ['tail_number', 'ICAO_code_origin', 'date', 'dep_UTC_time', 'tail_number_origin', 'ICAO_origin_origin', 'date_origin','departure_origin']].head()

,tail_number,ICAO_code_origin,date,dep_UTC_time,tail_number_origin,ICAO_origin_origin,date_origin,departure_origin
3413,N100KB,KGPI,2020-12-10,19:54:00,N100KB,KMOT,2020-12-06,19:54:00
3867,N100KB,KMOT,2020-12-08,14:20:00,N100KB,KDVL,2020-12-08,20:56:00
5870,N149CF,KSAT,2021-02-20,13:19:00,N149CF,KBMQ,2021-02-03,19:53:00
14364,N149CF,KINJ,2020-12-29,11:51:00,N149CF,KSAT,2020-12-29,11:51:00
15910,N149CF,KTKI,2020-12-15,18:08:00,N149CF,KHOU,2020-12-13,18:08:00


In [324]:
# drop this records
fw.loc[(fw.ICAO_code_origin.notnull()) & (fw.ICAO_origin_origin.notnull()) & (fw.ICAO_code_origin != fw.ICAO_origin_origin), fw.columns[-5:39]] = np.nan

In [325]:
# number of mismached date
fw.loc[(fw.date != fw.date_origin) & (fw.date_origin.notnull())].shape

(1260, 39)

In [326]:
# observe this records 
fw.loc[(fw.date != fw.date_origin) & (fw.date_origin.notnull()),
    ['tail_number', 'ICAO_code_origin', 'date', 'dep_UTC_time', 'tail_number_origin', 'ICAO_origin_origin', 'date_origin','departure_origin']].head()

,tail_number,ICAO_code_origin,date,dep_UTC_time,tail_number_origin,ICAO_origin_origin,date_origin,departure_origin
658,N100KB,KMOT,2021-02-09,14:20:00,N100KB,KMOT,2021-02-03,20:54:00
2020,N100KB,KXWA,2020-12-22,22:13:00,N100KB,KXWA,2020-12-13,01:19:00
3410,N100KB,KMOT,2020-12-13,00:21:00,N100KB,KMOT,2020-12-21,13:54:00
7376,N149CF,KGPM,2021-02-09,16:55:00,N149CF,KGPM,2020-12-09,04:55:00
13908,N149CF,KTKI,2021-01-03,17:51:00,N149CF,KTKI,2021-01-07,22:51:00


In [327]:
# drop this records
fw.loc[(fw.date != fw.date_origin) & (fw.date_origin.notnull()), fw.columns[-5:39]] = np.nan

In [458]:
# how much we lost
fw.METAR_origin.isna().sum()

6348

In [329]:
# drop duplicated columns
fw = fw.drop(['tail_number_origin', 'ICAO_origin_origin', 'date_origin', 'departure_origin'], axis=1)

In [330]:
# reset the indexes
fw = fw.reset_index(drop=True)

In [360]:
# Decode origin METAR data 
for i in range(len(fw)):
    try:
        code = fw.METAR_origin.iloc[i]
        obs = Metar.Metar(code)
        # The 'time' attribute is a datetime object
        fw.loc[i, 'METAR_time_origin'] = obs.time.ctime()
        # The report type attribute
        fw.loc[i, 'report_type_origin'] = obs.report_type()
        # The 'temp' and 'dewpt' attributes are temperature objects
        fw.loc[i, 'temperature_origin'] = obs.temp.string("F")
        fw.loc[i, 'dew_point_origin'] = obs.dewpt.string("F")
        # The wind() method returns a string describing wind observations
        # which may include speed, direction, variability and gusts.
        fw.loc[i, 'wind_origin'] = obs.wind()
        # The peak_wind() method returns a string describing the peak wind
        # speed and direction.
        fw.loc[i, 'wind_peak_origin'] = obs.peak_wind()
        # The visibility() method summarizes the visibility observation.
        fw.loc[i, 'visibility_origin'] = obs.visibility()
        # The 'press' attribute is a pressure object.
        fw.loc[i, 'pressure_origin'] = obs.press.string("mb")
        # The 'press' attribute is a pressure object.
        fw.loc[i, 'press_sea_level_origin'] = obs.press_sea_level.string("mb")
        # The sky_conditions() method summarizes the cloud-cover observations.
        fw.loc[i, 'sky_origin'] = obs.sky_conditions()
        # The remarks() method describes the remark groups that were parsed
        fw.loc[i, 'remarks_origin'] = obs.remarks()
    except:
        pass

In [363]:
# convert METAR_time_origin datetime
fw.METAR_time_origin = pd.to_datetime(fw.METAR_time_origin, format='%a %b %d %H:%M:%S %Y').dt.time

In [367]:
fw.head()

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,dep_time_zone,arrival,arr_time_zone,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,iata_code_origin,AIRPORT_NAME_origin,CITY_NAME_origin,COUNTRY_NAME_origin,STATE_NAME_origin,origin_state,ICAO_code_destination,iata_code_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,destination_state,dep_UTC_time,arr_UTC_time,METAR_origin,METAR_time_origin,report_type_origin,temperature_origin,dew_point_origin,wind_origin,wind_peak_origin,visibility_origin,pressure_origin,press_sea_level_origin,sky_origin,remarks_origin
0,N100KB,2021-02-20,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",16:57:00,CST,17:44:00,CST,0:46,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,46.772701,-100.746002,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND,22:57:00,23:44:00,METAR KXWA 202251Z AUTO 26013KT 10SM CLR 04/M0...,22:51:00,"routine report, cycle 23 (automatic report)",39.9 F,19.0 F,W at 13 knots,missing,10 miles,1008.5 mb,1012.1 mb,clear,Automated station (type 2)
1,N100KB,2021-02-20,BE9L,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,13:36:00,CST,14:27:00,CST,0:51,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",46.772701,-100.746002,48.260833,-103.751111,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,19:36:00,20:27:00,METAR KBIS 201952Z 16004KT 10SM CLR 06/M04 A29...,19:52:00,"routine report, cycle 20 (automatic report)",43.0 F,24.1 F,SSE at 4 knots,missing,10 miles,1010.2 mb,1012.3 mb,clear,Automated station (type 2)
2,N100KB,2021-02-18,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",08:20:00,CST,08:53:00,CST,0:32,105.704153,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,46.772701,-100.746002,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND,14:20:00,14:53:00,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N100KB,2021-02-15,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",14:15:00,CST,14:49:00,CST,0:33,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,48.259399,-101.279999,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,20:15:00,20:49:00,METAR KXWA 152051Z AUTO 15017G30KT 10SM CLR M1...,20:51:00,"routine report, cycle 21 (automatic report)",10.0 F,-9.9 F,"SSE at 17 knots, gusting to 30 knots",SE at 32 knots at 20:32,10 miles,1014.6 mb,1022.0 mb,clear,Automated station (type 2); peak wind 32kt fro...
4,N100KB,2021-02-15,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,08:00:00,CST,08:32:00,CST,0:32,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.260833,-103.751111,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,14:00:00,14:32:00,METAR KMOT 151454Z 14003KT 10SM CLR M26/M31 A3...,14:54:00,"rout

In [39]:
# # =======================================================================================
# #                       METAR data merging preparation for Destination
# # =======================================================================================
# prepare dataframe for merging
fw_destination = fw.loc[:,['tail_number','ICAO_code_destination', 'date', 'arr_UTC_time']]
fw_destination.head()

,tail_number,ICAO_code_destination,date,arr_UTC_time
0,N100KB,KMOT,2021-02-10,03:22:00
1,N100KB,KFAR,2021-01-25,14:43:00
2,N100KB,KMOT,2020-12-14,07:10:00
3,N100KB,KBIS,2021-02-07,04:18:00
4,N100KB,KMOT,2021-01-23,04:15:00


In [40]:
# replace dual ICAO code
fw_destination['spare'] = fw_destination.ICAO_code_destination.str.split(" / ", expand=True, n=1)[1]
fw_destination.loc[:,'ICAO_code_destination'] = fw_destination.ICAO_code_destination.str.split(" / ", expand=True, n=1)[0]
fw_destination.loc[fw_destination.spare.notnull(),'ICAO_code_destination'] = fw_destination.loc[fw_destination.spare.notnull(), 'spare']
fw_destination = fw_destination.drop('spare', axis=1)

In [41]:
# drop null values
fw_destination = fw_destination.dropna()

In [42]:
# sort by tail number 
fw_destination = fw_destination.sort_values('tail_number').reset_index(drop=True)

In [43]:
fw_destination.head()

,tail_number,ICAO_code_destination,date,arr_UTC_time
0,N100KB,KMOT,2021-02-10,03:22:00
1,N100KB,KDVL,2020-12-08,12:25:00
2,N100KB,KXWA,2020-12-08,14:59:00
3,N100KB,KRST,2020-12-08,22:52:00
4,N100KB,KSTP,2020-12-10,11:15:00


In [44]:
# # =======================================================================================
# #                               METAR data scraping for Destination
# # =======================================================================================

# # create containers
# tail_number = []
# ICAO_destination = []
# date = []
# METAR = []

# for j in range(0,len(fw_destination)-1):
#     url = "http://www.ogimet.com/display_metars2.php?lang=en&lugar=" + fw_destination.ICAO_code_destination.iloc[j] \
#       + "&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=" + str(fw_destination.date.dt.year.iloc[j]) \
#       + "&mes=" + str(fw_destination.date.dt.month.iloc[j]) + "&day=" + str(fw_destination.date.dt.day.iloc[j]) \
#       +"&hora=" + str(pd.to_datetime(fw_destination.arr_UTC_time, format='%H:%M:%S').dt.hour.iloc[j]) + "&anof=" + str(fw_destination.date.dt.year.iloc[j]) \
#       + "&mesf="+ str(fw_destination.date.dt.month.iloc[j]) + "&dayf=" + str(fw_destination.date.dt.day.iloc[j]) \
#     + "&horaf=" + str(pd.to_datetime(fw_destination.arr_UTC_time, format='%H:%M:%S').dt.hour.iloc[j] + 1) + "&minf=59&send=send"

#     # BeautifulSoup Magic
#     uClient = uReq(url)
#     page_html = uClient.read()
#     uClient.close()
#     page_soup = soup(page_html, 'html.parser')
    
#     if fw_destination.tail_number.iloc[j] == fw_destination.tail_number.iloc[j+1]:
#         for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
#             tail_number += [fw_destination.tail_number.iloc[j]]
#             ICAO_destination += [fw_destination.ICAO_code_destination.iloc[j]]
#             date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
#             METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]

#     # if reaches the end
#     if j == len(fw_destination)-2:
#         url = "http://www.ogimet.com/display_metars2.php?lang=en&lugar=" + fw_destination.ICAO_code_destination.iloc[-1] \
#           + "&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=" + str(fw_destination.date.dt.year.iloc[-1]) \
#           + "&mes=" + str(fw_destination.date.dt.month.iloc[-1]) + "&day=" + str(fw_destination.date.dt.day.iloc[-1]) \
#           +"&hora=" + str(pd.to_datetime(fw_destination.arr_UTC_time, format='%H:%M:%S').dt.hour.iloc[-1]) + "&anof=" + str(fw_destination.date.dt.year.iloc[-1]) \
#           + "&mesf="+ str(fw_destination.date.dt.month.iloc[-1]) + "&dayf=" + str(fw_destination.date.dt.day.iloc[-1]) \
#           + "&horaf=" + str(pd.to_datetime(fw_destination.arr_UTC_time, format='%H:%M:%S').dt.hour.iloc[-1] + 1) + "&minf=59&send=send"
        
#         # BeautifulSoup Magic
#         uClient = uReq(url)
#         page_html = uClient.read()
#         uClient.close()
#         page_soup = soup(page_html, 'html.parser')
        
#         for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
#             tail_number += [fw_destination.tail_number.iloc[-1]]
#             ICAO_destination += [fw_destination.ICAO_code_destination.iloc[-1]]
#             date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
#             METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
        
#         pd.DataFrame(list(zip(tail_number, ICAO_destination, date, METAR)), columns= ['tail_number', 'ICAO_destination', 'date', 'METAR'])\
#         .to_csv('../Datasets/METAR/FW/destination/' + fw_destination.tail_number.iloc[-1] + '.csv', index=False) 
        
#     else:
#         for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
#             tail_number += [fw_destination.tail_number.iloc[j]]
#             ICAO_destination += [fw_destination.ICAO_code_destination.iloc[j]]
#             date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
#             METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
        
#         pd.DataFrame(list(zip(tail_number, ICAO_destination, date, METAR)), columns= ['tail_number', 'ICAO_destination', 'date', 'METAR'])\
#         .to_csv('../Datasets/METAR/FW/destination/' + fw_destination.tail_number.iloc[j] + '.csv', index=False) 
        
#         tail_number = []
#         ICAO_destination = []
#         date = []
#         METAR = []

In [45]:
# # concat all files
# all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]

# #combine all files in the list
# combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# #export to csv
# combined_csv.to_csv("FW_METAR_destination.csv", index=False, encoding='utf-8-sig')

In [410]:
# # =======================================================================================
# #                               METAR data merging for Destination
# # =======================================================================================
# load the destination METAR dataframe
metar_destination = pd.read_csv("..//Datasets/METAR/FW/destination/FW_METAR_destination.csv")

In [411]:
metar_destination.head()

,tail_number,ICAO_destination,date,METAR
0,N783AL,KAPA,04/01/2021 20:53,METAR KAPA 042053Z 18008KT 10SM FEW200 12/M15 ...
1,N783AL,KAPA,04/01/2021 19:53,METAR KAPA 041953Z 18010KT 10SM FEW150 12/M15 ...
2,N783AL,KCOS,04/01/2021 20:54,METAR KCOS 042054Z 18011KT 10SM SCT250 09/M08 ...
3,N783AL,KCOS,04/01/2021 19:54,METAR KCOS 041954Z 19007KT 10SM SCT250 08/M09 ...
4,N783AL,KAPA,03/01/2021 08:53,METAR KAPA 030853Z 26005KT 10SM FEW150 M02/M13...


In [412]:
# split time from the date and assign a new column
metar_destination['arrival'] = metar_destination.date.str.split(" ", n=1, expand=True)[1]

# remove time from the date columns
metar_destination.date = metar_destination.date.str.split(" ", n=1, expand=True)[0]

In [423]:
# convert date to datetime
metar_destination.date  = pd.to_datetime(metar_destination.date, format='%d/%m/%Y')

# conver time to datetime
metar_destination.arrival = pd.to_datetime(metar_destination.arrival, format='%H:%M').dt.time

In [424]:
metar_destination.head()

,tail_number,ICAO_destination,date,METAR,arrival
0,N783AL,KAPA,2021-01-04,METAR KAPA 042053Z 18008KT 10SM FEW200 12/M15 ...,20:53:00
1,N783AL,KAPA,2021-01-04,METAR KAPA 041953Z 18010KT 10SM FEW150 12/M15 ...,19:53:00
2,N783AL,KCOS,2021-01-04,METAR KCOS 042054Z 18011KT 10SM SCT250 09/M08 ...,20:54:00
3,N783AL,KCOS,2021-01-04,METAR KCOS 041954Z 19007KT 10SM SCT250 08/M09 ...,19:54:00
4,N783AL,KAPA,2021-01-03,METAR KAPA 030853Z 26005KT 10SM FEW150 M02/M13...,08:53:00


In [425]:
# convert to string fot the best matching
metar_destination.date = metar_destination.date.astype(str)
metar_destination.arrival = metar_destination.arrival.astype(str)

In [441]:
# merge origin METAR data 
fw = fuzzymatcher.fuzzy_left_join(fw, metar_destination.add_suffix('_destination'), left_on = ['tail_number', 'ICAO_code_destination', 'date', 'arr_UTC_time'], 
                                  right_on = ['tail_number_destination','ICAO_destination_destination','date_destination', 'arrival_destination']).iloc[:,3:]

In [442]:
# count of records where the ICAO_code_destination is null or not in metar_destination column
fw.loc[(~fw.ICAO_code_destination.isin(fw.ICAO_destination_destination)) | (fw.ICAO_code_destination.isna())].shape

(2865, 51)

In [443]:
# Observe where the ICAO_code_destination is null or not in metar_destination columns
fw.loc[(~fw.ICAO_code_destination.isin(fw.ICAO_destination_destination)) | (fw.ICAO_code_destination.isna()), ['ICAO_code_destination', 'ICAO_destination_destination']].head()

,ICAO_code_destination,ICAO_destination_destination
387,06D,KMOT
1562,S25,KMOT
1783,06D,KMOT
2565,NaN,KBIL
2773,NaN,KMOT


In [444]:
# Drop these records
fw.loc[(~fw.ICAO_code_destination.isin(fw.ICAO_destination_destination)) | (fw.ICAO_code_destination.isna()), fw.columns[-5:52]] = np.nan

In [445]:
# number of mismached tail numbers
fw.loc[(fw.tail_number != fw.tail_number_destination) & (fw.tail_number_destination.notnull())].shape

(808, 51)

In [446]:
# inspect this records
fw.loc[(fw.tail_number != fw.tail_number_destination) & (fw.tail_number_destination.notnull()),
       ['tail_number', 'ICAO_code_destination', 'date', 'arr_UTC_time', 'tail_number_destination', 'ICAO_destination_destination', 'date_destination','arrival_destination']].head()

,tail_number,ICAO_code_destination,date,arr_UTC_time,tail_number_destination,ICAO_destination_destination,date_destination,arrival_destination
4424,N100KB,KMOT,2020-12-08,13:21:00,N888CP,KDAL,2020-12-08,21:53:00
7176,N149CF,KOKC,2021-02-11,05:35:00,N783AL,KCAG,2021-01-02,05:35:00
12975,N149CF,KHOB,2021-01-04,19:56:00,N783AL,KAPA,2021-01-04,19:53:00
13121,N149CF,KHOU,2021-01-04,12:07:00,N237PC,KHOU,2021-01-04,07:53:00
17887,N17NM,KABQ,2021-02-19,11:05:00,N642PC,KABQ,2021-02-19,05:52:00


In [447]:
# Drop these records
fw.loc[(fw.tail_number != fw.tail_number_destination) & (fw.tail_number_destination.notnull()), fw.columns[-5:52]] = np.nan

In [448]:
# number of mismached ICAO_destination_destination
fw.loc[(fw.ICAO_code_destination.notnull()) & (fw.ICAO_destination_destination.notnull()) & (fw.ICAO_code_destination != fw.ICAO_destination_destination)].shape

(734, 51)

In [449]:
# inspect this records
fw.loc[(fw.ICAO_code_destination.notnull()) & (fw.ICAO_destination_destination.notnull()) & (fw.ICAO_code_destination != fw.ICAO_destination_destination),
       ['tail_number', 'ICAO_code_destination', 'date', 'arr_UTC_time', 'tail_number_destination', 'ICAO_destination_destination', 'date_destination','arrival_destination']].head()

,tail_number,ICAO_code_destination,date,arr_UTC_time,tail_number_destination,ICAO_destination_destination,date_destination,arrival_destination
4314,N100KB,KSTP,2020-12-10,11:15:00,N100KB,KMOT,2020-12-10,15:54:00
4368,N100KB,KRST,2020-12-08,22:52:00,N100KB,KBIS,2020-12-31,22:52:00
5505,N149CF,KGPM,2021-02-26,23:05:00,N149CF,KGDJ,2020-12-26,05:15:00
6708,N149CF,KTKI,2021-02-19,02:15:00,N149CF,KGPM,2021-01-19,02:15:00
7124,N149CF,KGPM,2021-02-12,09:08:00,N149CF,KTKI,2021-02-12,08:53:00


In [450]:
# Drop these records
fw.loc[(fw.ICAO_code_destination.notnull()) & (fw.ICAO_destination_destination.notnull()) & (fw.ICAO_code_destination != fw.ICAO_destination_destination), fw.columns[-5:52]] = np.nan

In [451]:
# number of mismached date
fw.loc[(fw.date != fw.date_destination) & (fw.date_destination.notnull())].shape

(1364, 51)

In [452]:
fw.loc[(fw.date != fw.date_destination) & (fw.date_destination.notnull()),
        ['tail_number', 'ICAO_code_destination', 'date', 'arr_UTC_time', 'tail_number_destination', 'ICAO_destination_destination', 'date_destination','arrival_destination']].head()

,tail_number,ICAO_code_destination,date,arr_UTC_time,tail_number_destination,ICAO_destination_destination,date_destination,arrival_destination
189,N100KB,KMOT,2021-02-10,03:22:00,N100KB,KMOT,2020-12-10,22:54:00
609,N100KB,KMOT,2021-02-08,08:13:00,N100KB,KMOT,2020-12-08,13:54:00
990,N100KB,KMOT,2021-02-03,00:04:00,N100KB,KMOT,2021-02-10,04:54:00
3012,N100KB,KXWA,2020-12-28,14:22:00,N100KB,KXWA,2020-12-22,14:51:00
6709,N149CF,KGPM,2021-02-19,23:27:00,N149CF,KGPM,2021-02-27,00:15:00


In [453]:
# Drop these records
fw.loc[(fw.date != fw.date_destination) & (fw.date_destination.notnull()), fw.columns[-5:52]] = np.nan

In [457]:
# how much we lost
fw.METAR_destination.isna().sum()

5771

In [454]:
# drop duplicated columns
fw = fw.drop(['tail_number_destination', 'ICAO_destination_destination', 'date_destination', 'arrival_destination'], axis=1)

In [455]:
# reset the indexes
fw = fw.reset_index(drop=True)

In [456]:
# Decode destination METAR data 
for i in range(len(fw)):
    try:
        code = fw.METAR_destination.iloc[i]
        obs = Metar.Metar(code)
        # The 'time' attribute is a datetime object
        fw.loc[i, 'METAR_time_destination'] = obs.time.ctime()
        # The report type attribute
        fw.loc[i, 'report_type_destination'] = obs.report_type()
        # The 'temp' and 'dewpt' attributes are temperature objects
        fw.loc[i, 'temperature_destination'] = obs.temp.string("F")
        fw.loc[i, 'dew_point_destination'] = obs.dewpt.string("F")
        # The wind() method returns a string describing wind observations
        # which may include speed, direction, variability and gusts.
        fw.loc[i, 'wind_destination'] = obs.wind()
        # The peak_wind() method returns a string describing the peak wind
        # speed and direction.
        fw.loc[i, 'wind_peak_destination'] = obs.peak_wind()
        # The visibility() method summarizes the visibility observation.
        fw.loc[i, 'visibility_destination'] = obs.visibility()
        # The 'press' attribute is a pressure object.
        fw.loc[i, 'pressure_destination'] = obs.press.string("mb")
        # The 'press' attribute is a pressure object.
        fw.loc[i, 'press_sea_level_destination'] = obs.press_sea_level.string("mb")
        # The sky_conditions() method summarizes the cloud-cover observations.
        fw.loc[i, 'sky_destination'] = obs.sky_conditions()
        # The remarks() method describes the remark groups that were parsed
        fw.loc[i, 'remarks_destination'] = obs.remarks()
    except:
        pass

In [459]:
# convert METAR_time_origin datetime
fw.METAR_time_destination = pd.to_datetime(fw.METAR_time_destination, format='%a %b %d %H:%M:%S %Y').dt.time

In [460]:
fw.head()

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,dep_time_zone,arrival,arr_time_zone,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,iata_code_origin,AIRPORT_NAME_origin,CITY_NAME_origin,COUNTRY_NAME_origin,STATE_NAME_origin,origin_state,ICAO_code_destination,iata_code_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,destination_state,dep_UTC_time,arr_UTC_time,METAR_origin,METAR_time_origin,report_type_origin,temperature_origin,dew_point_origin,wind_origin,wind_peak_origin,visibility_origin,pressure_origin,press_sea_level_origin,sky_origin,remarks_origin,METAR_destination,METAR_time_destination,report_type_destination,temperature_destination,dew_point_destination,wind_destination,wind_peak_destination,visibility_destination,pressure_destination,press_sea_level_destination,sky_destination,remarks_destination
0,N100KB,2021-02-20,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",16:57:00,CST,17:44:00,CST,0:46,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,46.772701,-100.746002,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND,22:57:00,23:44:00,METAR KXWA 202251Z AUTO 26013KT 10SM CLR 04/M0...,22:51:00,"routine report, cycle 23 (automatic report)",39.9 F,19.0 F,W at 13 knots,missing,10 miles,1008.5 mb,1012.1 mb,clear,Automated station (type 2),METAR KBIS 202352Z 08004KT 10SM CLR 04/M04 A29...,23:52:00,"routine report, cycle 24 (automatic report)",39.0 F,24.1 F,E at 4 knots,missing,10 miles,1010.5 mb,1012.8 mb,clear,Automated station (type 2); 3-hr pressure chan...
1,N100KB,2021-02-20,BE9L,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,13:36:00,CST,14:27:00,CST,0:51,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",46.772701,-100.746002,48.260833,-103.751111,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,19:36:00,20:27:00,METAR KBIS 201952Z 16004KT 10SM CLR 06/M04 A29...,19:52:00,"routine report, cycle 20 (automatic report)",43.0 F,24.1 F,SSE at 4 knots,missing,10 miles,1010.2 mb,1012.3 mb,clear,Automated station (type 2),METAR KXWA 202051Z AUTO 23012KT 10SM CLR 03/M0...,20:51:00,"routine report, cycle 21 (automatic report)",37.9 F,21.0 F,SW at 12 knots,missing,10 miles,1008.5 mb,1012.1 mb,clear,Automated station (type 2); 3-hr pressure chan...
2,N100KB,2021-02-18,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",08:20:00,CST,08:53:00,CST,0:32,105.704153,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,46.772701,-100.746002,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND,14:20:00,14:53:00,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,METAR KBIS 181452Z 28007KT 10SM OVC070 M14/M18...,14:52:00,"routine report, cycle 15 (automatic report)",7.0 F,-0.9 F,W at 7 knots,missing,10 miles,1026.1 mb,1030.1 mb,overcast at 7000 feet,Automated station (type 2); 3-hr pressure chan...
3,N100KB,2021-02-15,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",14:15:00,CST,14:49:00,CST,0:33,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,48.259399,-101.279999,KXWA,XWA,Williston Basin International,Williston - 

In [461]:
fw.isna().sum()

tail_number                       0
date                              0
aircraft                        190
origin                            0
origin_location                   0
destination                       0
destination_location              0
departure                         0
dep_time_zone                     0
arrival                           0
arr_time_zone                     0
duration                         20
distance_mi                     301
Owner                             0
origin_Latitude                 156
origin_Longitude                156
destination_Latitude            147
destination_Longitude           147
ICAO_code_origin                873
iata_code_origin               1011
AIRPORT_NAME_origin            1069
CITY_NAME_origin               1069
COUNTRY_NAME_origin            1069
STATE_NAME_origin              1159
origin_state                     32
ICAO_code_destination           870
iata_code_destination          1007
AIRPORT_NAME_destination    

In [462]:
fw.to_csv('../Datasets/NEW/FW_with_metar.csv', index=False)